In [1]:
%load_ext autoreload
%autoreload 2

import torch
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/ubuntu/focus-convolutional-neural-network/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import of clean dataloader

In [2]:
from data_utils.data_loaders import FocusCNNLoader
dl = FocusCNNLoader(
    images_dir='/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/images',
    batch_size=16,
    csv_path='/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/labels.csv',
    save_out_dir = "/home/ubuntu/focus-convolutional-neural-network/res/focus_cnn/coco_focuscnn/trainer/labels/",
    shuffle=True,
    validation_split=0.15,
    num_workers=12,
    labels={
        "0": "none",
        "1": "person",
        "2": "car",
        "3": "bicycle"
    },
    tf_image_size=(640,640),
    is_test=False
)
dl_train = dl.get_train_loader()

# Function to calculate mean and std of the dataset

In [9]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data in tqdm(dataloader, desc="Calculating mean and std"):
        # print(data)
        # Mean over batch, height and width, but not over the channels
        data = data[0]
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [10]:
get_mean_and_std(dl_train)

Calculating mean and std: 100%|██████████| 6284/6284 [12:05<00:00,  8.66it/s]


(tensor([0.4615, 0.4410, 0.4051]), tensor([0.2699, 0.2703, 0.2797]))

# Check if works :) 

In [11]:
from data_utils.data_loaders import FocusCNNLoader
dl = FocusCNNLoader(
    images_dir='/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/images',
    batch_size=16,
    csv_path='/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/labels.csv',
    save_out_dir = "/home/ubuntu/focus-convolutional-neural-network/res/focus_cnn/coco_focuscnn/trainer/labels/",
    shuffle=True,
    validation_split=0.15,
    num_workers=12,
    labels={
        "0": "none",
        "1": "person",
        "2": "car",
        "3": "bicycle"
    },
    tf_image_size=(640,640),
    is_test=False,
    transform_mean=[0.4615, 0.4410, 0.4051],
    transform_std=[0.2699, 0.2703, 0.2797],
)
dl_train = dl.get_train_loader()

In [12]:
get_mean_and_std(dl_train)

Calculating mean and std: 100%|██████████| 6284/6284 [12:01<00:00,  8.70it/s]


(tensor([-0.0007,  0.0003,  0.0003]), tensor([0.9992, 0.9997, 1.0002]))

In [13]:
data = next(iter(dl_train))

In [21]:
data[1]

tensor([ 22884.,  72394., 113038.,  34670.,  24835.,  73844.,  54118.,  29080.,
        113907.,   8121.,   5811.,  97680.,  27994.,  80958.,  35899.,  22886.])

In [22]:
data[2]

{'1': {'label': tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.]),
  'transform': tensor([[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.2303, -0.2058, -2.1012],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.2365,  0.2876, -1.1026],
          [ 0.1757, -0.0051, -0.1973],
          [ 0.0097,  0.0300, -0.1561],
          [ 0.2721,  0.0695, -2.1217],
          [ 0.2105, -0.1224, -0.7124],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]])},
 '2': {'label': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.]),
  'transform': tensor([[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [

# Classifiers:
## Person:
+ "transform_mean": [0.5462,0.5803,0.6043]
+ "transform_std": [0.2952,0.2886,0.2965]
## Car:
+ "transform_mean": [0.5615,0.5820,0.5965]
+ "transform_std": [0.2911,0.2849,0.2908]
## Dog:
+ "transform_mean": [0.5243,0.5564,0.5869],
+ "transform_std": [0.2958,0.2923,0.3011]
## Bicycle:
+ "transform_mean": [0.5647,0.5906,0.6088]
+ "transform_std": [0.2894,0.283,0.2891]
## Cat:
+ "transform_mean": [0.5295,0.5672,0.5936]
+ "transform_std": [0.3045,0.3015,0.3107]
## Multi:
+ "transform_mean": [0.5438,0.5786,0.6030]
+ "transform_std": [0.2944,0.2882,0.2966]

# Focus:
## Person:
+ "transform_mean": [0.4686, 0.4496, 0.4134]
+ "transform_std": [0.2553, 0.2530, 0.2708]
## Car:
+ "transform_mean": [0.4572, 0.4423, 0.4103]
+ "transform_std": [0.2577, 0.2591, 0.2712]

# Class weights computing

In [1]:
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
class_counts = torch.bincount(target)
total_samples = class_counts.sum()
class_weights = total_samples / (num_classes * class_counts)

In [2]:
df_train = pd.read_csv("/home/ubuntu/focus-convolutional-neural-network/res/focus_cnn/coco_focuscnn/trainer/labels/labels_train.csv")
# df_train_aug = pd.read_csv("/home/ubuntu/focus-convolutional-neural-network/res/classifiers/coco_classifier_multi/trainer/labels/labels_train_aug.csv")

In [7]:
print("1", df_train['label_1'].apply(lambda x: 1 if x == 1 else 0).sum())
print("0", df_train['label_1'].apply(lambda x: 1 if x == 0 else 0).sum())
print("2", df_train['label_2'].apply(lambda x: 1 if x == 2 else 0).sum())
print("0", df_train['label_2'].apply(lambda x: 1 if x == 0 else 0).sum())
print("3", df_train['label_3'].apply(lambda x: 1 if x == 3 else 0).sum())
print("0", df_train['label_3'].apply(lambda x: 1 if x == 0 else 0).sum())

1 54391
0 46152
2 10361
0 90182
3 2733
0 97810


In [8]:
46152 + 90182 + 97810

234144

In [10]:
import torch

In [11]:
class_counts = [234144, 54391, 10361, 2733]
target = torch.tensor([0, 1, 2, 3])

# Calculate the weights for each class
total_samples = sum(class_counts)
class_weights = torch.tensor([total_samples / (4 * count) for count in class_counts])
class_weights

tensor([ 0.3221,  1.3864,  7.2780, 27.5914])

In [3]:
df_train['label'].value_counts()

KeyError: 'label'

In [32]:
df_train_aug['label'].value_counts()

3    5335
2     865
Name: label, dtype: int64

In [33]:
list_train = df_train['label'].to_list()

In [34]:
list_train_aug = df_train_aug['label'].to_list()

In [35]:
list_train_full = list_train + list_train_aug

In [36]:
compute_class_weight(class_weight='balanced', classes=np.unique(list_train_full), y=list_train_full)

array([ 0.39715357,  0.82703406,  4.82156079, 15.25773196])